### Install winbind components

In [ ]:
yum install samba-winbind-clients samba-winbind samba-common-tools samba-winbind-krb5-locator

### Install Samba

In [ ]:
yum install samba

### Enable SMB and NMB

In [ ]:
systemctl enable --now smb
systemctl enable --now nmb

### visudo (add AD groups to sudoers file with NETBIOS domain syntax) i.e.
### Allows people in group wheel to run all commands
```bash
%wheel ALL=(ALL) ALL
%Local\ Admins-Linux\ Servers@contoso.com ALL=(ALL) NOPASSWD: ALL
%SERVER1-Local\ Admin@contoso.com ALL=(ALL) NOPASSWD: ALL
%NETBIOS\\Local\ Admins-Linux\ Servers ALL=(ALL) NOPASSWD: ALL
%NETBIOS\\SERVER1-Local\ Admin ALL=(ALL) NOPASSWD: ALL
```

### Leave Realm

In [ ]:
realm leave --user=username@contoso.com contoso.com

### Join Realm

In [ ]:
realm join --client-software=winbind --user=username@contoso.com contoso.com

### Add user to local Linux Group
### use
cat /etc/group to see

In [ ]:
usermod -aG swcc "CONTOSO\username"

### configure smb.conf file
```yaml
[global]
security = ads
passdb backend = tdbsam
# log files split per-machine:
log file = /var/log/samba/log.%m
# enable the following line to debug:
log level =3
# maximum size of 500KB per log file, then rotate:
max log size = 500
kerberos method = secrets and keytab
template homedir = /home/%U@%D
workgroup = NETBIOS
template shell = /bin/bash
realm = CONTOSO.COM #Change upn
idmap config NETBIOS : range = 2000000-2999999 #Change netbios
idmap config NETBIOS : backend = rid #Change netbios
idmap config * : range = 10000-999999
idmap config * : backend = tdb
winbind use default domain = no
winbind refresh tickets = yes
winbind offline logon = yes
winbind enum groups = no
winbind enum users = no

[capture]
# hosts allow = 10.74.0.28 10.74.0.136 10.70.2.34
path = /mnt/capture
guest ok = no
valid users = @"NETBIOS\ad_group_name" #Change AD Group Name, or User
writeable = yes
create mask = 0775
directory mask = 0775
```

#### Reload SMB config

In [ ]:
smbcontrol all reload-config